# Properly commented Code file with groupings, dataset and numeric computations

# IMPORT THE DATA

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

Define functions or classes if needed

In [2]:
 # Read dataset into a pandas DataFrame
    
dataset = pd.read_csv("Recruitment_Data.csv")

In [3]:
dataset

,attrition,performance_rating,sales_quota_pct,recruiting_source
0,1,3,1.088190,Applied Online
1,0,3,2.394173,NaN
2,1,2,0.497530,Campus
3,0,2,2.513958,NaN
4,0,3,1.424789,Applied Online
...,...,...,...,...
441,0,3,1.627813,Campus
442,1,4,0.689682,Referral
443,0,4,1.196198,NaN
444,0,4,1.598213,Applied Online


In [4]:
# Display basic information about the dataset

print("Dataset Information:")
print(dataset.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   attrition           446 non-null    int64  
 1   performance_rating  446 non-null    int64  
 2   sales_quota_pct     446 non-null    float64
 3   recruiting_source   241 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 14.1+ KB
None


Function for numeric computations def perform_numeric_computations(data): Perform numeric computations on the dataset.

In [5]:
dataset.shape

(446, 4)

In [6]:
dataset.describe()

,attrition,performance_rating,sales_quota_pct
count,446.000000,446.000000,446.000000
mean,0.213004,2.894619,1.082607
std,0.409891,0.683755,0.710042
min,0.000000,1.000000,-0.710827
25%,0.000000,2.000000,0.584444
50%,0.000000,3.000000,1.070143
75%,0.000000,3.000000,1.532488
max,1.000000,5.000000,3.666740


# Groupings the dataset 

In [7]:
# Grouping the data based on a categorical column, e.g., 'recruiting_source'
grouped_data = dataset.groupby('recruiting_source')

In [8]:
dataset.isna().sum()

attrition               0
performance_rating      0
sales_quota_pct         0
recruiting_source     205
dtype: int64

In [9]:
#Checking the unique values in recruitment_source column where we have missing values
dataset['recruiting_source'].unique()

array(['Applied Online', nan, 'Campus', 'Referral', 'Search Firm'],
      dtype=object)

In [10]:
#Checking if the dataset contains duplicate values
dataset.duplicated().sum()

0

In [11]:
missing_perct=round(dataset['recruiting_source'].isna().sum()/len(dataset)*100)
print(f"There is  {missing_perct}% data missing from recruiting_source")

There is  46% data missing from recruiting_source


 Imputing the missing values with knn bfill or summary statistics is a good option
 
 My intution is these missing data has been recorded and have no association with other catgories
 
 Here i think that as 46 % of values are missing there is a higher chance of underfitting and bias(noise)
 
 I will be imputing these mssing values as Walk in or direct interviews which make
 

In [12]:
#Lets Take NA values as Walk In
dataset.fillna("Walk In",inplace=True)

In [13]:
#Before Removing the Na values leets group the dataset with the NA values present and see if it checks out
grouped=dataset.groupby("recruiting_source").agg(
    total_attrition=("attrition","sum"),
    total_recruites=("attrition","count"),
    average_sales=("sales_quota_pct","mean")
).reset_index()
grouped['average_attrition']=round(grouped['total_attrition']/grouped['total_recruites']*100,2)

In [14]:
'''
*Most of the recruits by the company is Walkin followed by online recurits
*These Walk In in hires also have the highest averge_sales followed by online recurits
*Walk In interviews have the most less average attrition followed by online recuits
'''
#We will sort the dataframe to find the recuriting_source with lowest attrition  rate and high average sales
grouped.sort_values(by=["average_sales","average_attrition"],ascending=[0,1])

,recruiting_source,total_attrition,total_recruites,average_sales,average_attrition
4,Walk In,27,205,1.168109,13.17
0,Applied Online,32,130,1.058590,24.62
2,Referral,15,45,1.023198,33.33
1,Campus,16,56,0.908035,28.57
3,Search Firm,5,10,0.886960,50.00


# Numeric Computations

In [15]:
# Compute mean, median, and sum for numeric columns within each group
computed_results = grouped_data.agg({
        'attrition': 'mean',
        'performance_rating': ['mean', 'median', 'sum'],
        'sales_quota_pct': ['mean', 'median', 'sum']
}).reset_index()

In [16]:
computed_results

recruiting_source attrition performance_rating             sales_quota_pct  \
                         mean               mean median  sum            mean   
0    Applied Online  0.246154           2.930769    3.0  381        1.058590   
1            Campus  0.285714           2.928571    3.0  164        0.908035   
2          Referral  0.333333           2.844444    3.0  128        1.023198   
3       Search Firm  0.500000           2.700000    3.0   27        0.886960   
4           Walk In  0.131707           2.882927    3.0  591        1.168109   

                         
     median         sum  
0  1.035413  137.616724  
1  0.918977   50.849983  
2  0.982009   46.043918  
3  1.276763    8.869603  
4  1.204520  239.462374